In [31]:
import os
import pandas as pd
from src.utils import data_load
from datetime import datetime, timedelta
import numpy as np
from sklearn.linear_model import LinearRegression

In [40]:
root_path = '/mnt/results'
src_dir = 'edema_pred_window'
dst_dir = 'edema_pred_window_sorted_weight_normalized'
dst_dir = 'edema_pred_window_sorted'
window_size = 21

In [6]:
df_birth = data_load(data_keys={"birth"}, wave=5)['birth']

In [41]:
for f in os.listdir(os.path.join(root_path, src_dir)):
    if 'date' in f:
        # figure out user id
        user_id = int(f.split('_')[1])
        
        # figure out first day of third trimester
        curr_birth_date = pd.to_datetime(df_birth[df_birth.user_id == user_id].birth_date.values[0])
        third_trimester_start_date = curr_birth_date - timedelta(days=91)
        # figure out four slices start date
        snd_start_date = third_trimester_start_date + timedelta(days=window_size)
        trd_start_date = snd_start_date + timedelta(days=window_size)
        fth_start_date = trd_start_date + timedelta(days=window_size)
        # figure out start day of the slice
        curr_df = pd.read_csv(os.path.join(root_path, src_dir, f))
        first_day_curr_df = pd.to_datetime(curr_df.date.values[0])
        
        # compare
        ckpt_dates = [third_trimester_start_date, snd_start_date, trd_start_date, fth_start_date]
        dist_to_ckpt_dates = list(map(lambda x: abs((x-first_day_curr_df).days), ckpt_dates))
        
        closest_ckpt_idx = np.argmin(dist_to_ckpt_dates)
        if 'weight' in dst_dir:
            weight_ys = np.array(curr_df.weight_kg.tolist()).reshape(-1, 1)
            weight_xs = np.array(list(range(window_size))).reshape(-1, 1)

            weight_xs_train = weight_xs[:14, :]
            weight_ys_train = weight_ys[:14, :]

            weight_xs_test = weight_xs[14:, :]
            weight_ys_test = weight_ys[14:, :]

            linear_model = LinearRegression()
            linear_model.fit(weight_xs_train, weight_ys_train)
            
            pred = linear_model.predict(weight_xs)
            residual = weight_ys - pred
            residual = residual.reshape(-1,)
            
            curr_df.weight_kg = residual
        
        save_df = curr_df.drop('date', axis=1)
        save_df.to_csv(os.path.join(root_path, dst_dir, f'user_{user_id}_{dst_dir}_slice{closest_ckpt_idx}.csv'), index=False, header=False)